# ArcGIS Connector - Wide to Long Transformation

Этот ноутбук:
1. Читает данные из Google Sheets
2. Трансформирует из Wide to Long формат
3. Загружает данные в ArcGIS Feature Layer


In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


## Шаг 1: Настройка доступа к Google Sheets

**Вариант 1: OAuth (проще для начала)**
- При первом запуске откроется браузер для авторизации
- Разрешите доступ к Google Sheets

**Вариант 2: Service Account**
- Создайте Service Account в Google Cloud Console
- Скачайте JSON ключ и сохраните как `credentials.json`
- Поделитесь Google Sheet с email из Service Account


In [ ]:
# ВАРИАНТ 1: OAuth (откроется браузер для авторизации)
gc = gspread.oauth()

# ВАРИАНТ 2: Service Account (раскомментируйте если используете)
# SCOPE = [
#     'https://www.googleapis.com/auth/spreadsheets',
#     'https://www.googleapis.com/auth/drive'
# ]
# creds = Credentials.from_service_account_file('credentials.json', scopes=SCOPE)
# gc = gspread.authorize(creds)

# URL вашего Google Sheet
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/12846JbH2PwR0wN8eLVnosg4xujw-04gKyyD6RuElc-4/edit?gid=0#gid=0'

# Открываем таблицу
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1  # или sh.worksheet('Sheet1')

print(f"Таблица открыта: {sh.title}")


## Шаг 2: Чтение данных из Google Sheets


In [ ]:
# Читаем все данные
data = worksheet.get_all_records()
df = pd.DataFrame(data)

print(f"Загружено строк: {len(df)}")
print(f"Колонки: {list(df.columns)}")
df.head()


## Шаг 3: Трансформация Wide to Long

Преобразуем колонки 'Значення 1' - 'Значення 10' в длинный формат. 
Согласно ТЗ, каждая строка с заполненными значениями должна быть развернута в отдельные строки.


In [ ]:
# Определяем колонки для нормализации
value_columns = [col for col in df.columns if 'Значення' in col or 'value' in col.lower()]

# Колонки, которые остаются (ID колонки)
id_columns = [col for col in df.columns if col not in value_columns and col not in ['long', 'lat']]

print(f"Колонки для нормализации: {value_columns}")
print(f"ID колонки (остаются): {id_columns}")


In [ ]:
# Трансформация Wide to Long
# Создаем список для хранения развернутых строк
expanded_rows = []

for idx, row in df.iterrows():
    # Для каждой строки создаем отдельные строки для каждого заполненного значения
    for value_col in value_columns:
        if pd.notna(row[value_col]) and row[value_col] != '' and row[value_col] != 0:
            # Создаем новую строку
            new_row = row[id_columns + ['long', 'lat']].copy()
            # Добавляем все значения (для соответствия схеме ArcGIS)
            for vc in value_columns:
                new_row[vc] = row[vc] if pd.notna(row[vc]) else 0
            expanded_rows.append(new_row)

# Создаем новый DataFrame
if expanded_rows:
    df_long = pd.DataFrame(expanded_rows)
else:
    # Если нет развернутых строк, используем исходный формат
    df_long = df.copy()

print(f"После трансформации: {len(df_long)} строк")
df_long.head(10)


## Шаг 4: Подготовка данных для ArcGIS

Маппинг полей согласно ТЗ


In [ ]:
# Маппинг полей согласно ТЗ
field_mapping = {
    'd_date': 'Дата',
    't_region': 'Область',
    't_city': 'Місто',
    'i_value_1': 'Значення 1',
    'i_value_2': 'Значення 2',
    'i_value_3': 'Значення 3',
    'i_value_4': 'Значення 4',
    'i_value_5': 'Значення 5',
    'i_value_6': 'Значення 6',
    'i_value_7': 'Значення 7',
    'i_value_8': 'Значення 8',
    'i_value_9': 'Значення 9',
    'i_value_10': 'Значення 10',
    'long': 'long',
    'lat': 'lat'
}

# Переименовываем колонки согласно маппингу
df_final = df_long.copy()
for arcgis_field, source_field in field_mapping.items():
    if source_field in df_final.columns:
        df_final = df_final.rename(columns={source_field: arcgis_field})

# Проверяем наличие обязательных полей
required_fields = ['long', 'lat', 'd_date', 't_region', 't_city']
missing_fields = [f for f in required_fields if f not in df_final.columns]
if missing_fields:
    print(f"ВНИМАНИЕ: Отсутствуют поля: {missing_fields}")
else:
    print("✅ Все обязательные поля присутствуют")

print(f"\nКолонки в финальном DataFrame: {list(df_final.columns)}")
df_final.head()


## Шаг 10: Получить ссылку на карту


In [ ]:
# Получаем ссылку на карту несколькими способами

# Способ 1: Через item (если переменная item доступна)
try:
    if 'item' in globals():
        print("="*60)
        print("📍 ССЫЛКИ НА КАРТУ:")
        print("="*60)
        print(f"\n1. Item URL: {item.url}")
        print(f"\n2. Item ID: {item.id}")
        print(f"\n3. Прямая ссылка на карту: https://www.arcgis.com/home/item.html?id={item.id}")
        
        # Если это Feature Service, получаем ссылку на веб-карту
        if hasattr(item, 'url'):
            print(f"\n4. Feature Service URL: {item.url}")
        
        # Пытаемся найти связанную веб-карту
        if hasattr(item, 'relatedItems'):
            related = item.relatedItems()
            for rel_item in related:
                if rel_item.type == 'Web Map':
                    print(f"\n5. Web Map URL: {rel_item.url}")
                    print(f"   Web Map ID: {rel_item.id}")
except NameError:
    pass

# Способ 2: Через Feature Layer ID напрямую
FEATURE_LAYER_ID = '6f8eb77db2dc41cc8261eb836e4b35af'
print(f"\n6. Прямая ссылка через ID: https://www.arcgis.com/home/item.html?id={FEATURE_LAYER_ID}")

# Способ 3: Через ArcGIS Online напрямую
print(f"\n7. Откройте в ArcGIS Online: https://www.arcgis.com/apps/mapviewer/index.html?layers={FEATURE_LAYER_ID}")

print("\n" + "="*60)
print("💡 ИНСТРУКЦИЯ:")
print("="*60)
print("1. Скопируйте любую из ссылок выше")
print("2. Вставьте в браузер")
print("3. Войдите в свой ArcGIS аккаунт (если потребуется)")
print("4. Вы увидите карту с вашими точками!")
print("="*60)


## ⚠️ ПРОБЛЕМА: Карта показывает весь мир, точки не видны

Если вы видите карту мира, но не видите точки, выполните эту ячейку для диагностики и исправления


In [ ]:
# ДИАГНОСТИКА И РЕШЕНИЕ ПРОБЛЕМЫ С КАРТОЙ

FEATURE_LAYER_ID = '6f8eb77db2dc41cc8261eb836e4b35af'

print("="*70)
print("🔍 ДИАГНОСТИКА: Почему не видно точек на карте")
print("="*70)

# Проверка 1: Есть ли подключение к ArcGIS
if 'gis' not in globals():
    print("\n❌ ПРОБЛЕМА: Не подключено к ArcGIS")
    print("💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")
else:
    print(f"\n✅ Подключение к ArcGIS: OK ({gis.properties.name})")
    
    # Проверка 2: Есть ли Feature Layer
    if 'feature_layer' not in globals():
        print("\n❌ ПРОБЛЕМА: Feature Layer не загружен")
        print("💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 6: Получение Feature Layer'")
    else:
        print("\n✅ Feature Layer: OK")
        
        # Проверка 3: Есть ли данные в слое
        try:
            count = feature_layer.query(where="1=1", return_count_only=True)
            print(f"\n📊 Количество features в слое: {count}")
            
            if count == 0:
                print("\n❌ ПРОБЛЕМА: В слое НЕТ данных!")
                print("💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 8: Загрузка данных в Feature Layer'")
            else:
                print("\n✅ Данные в слое: ЕСТЬ")
                
                # Получаем границы данных для масштабирования
                try:
                    extent_result = feature_layer.query(where="1=1", return_extent_only=True)
                    if extent_result and 'extent' in extent_result:
                        extent = extent_result['extent']
                        xmin = extent.get('xmin', 0)
                        xmax = extent.get('xmax', 0)
                        ymin = extent.get('ymin', 0)
                        ymax = extent.get('ymax', 0)
                        
                        print(f"\n📍 Границы ваших данных:")
                        print(f"   Долгота: {xmin:.6f} до {xmax:.6f}")
                        print(f"   Широта: {ymin:.6f} до {ymax:.6f}")
                        
                        # Создаем ссылку с правильным масштабированием
                        center_x = (xmin + xmax) / 2
                        center_y = (ymin + ymax) / 2
                        
                        print("\n" + "="*70)
                        print("🔗 ССЫЛКИ НА КАРТУ С ПРАВИЛЬНЫМ МАСШТАБОМ:")
                        print("="*70)
                        
                        # Ссылка 1: Map Viewer с центром и зумом
                        zoom_level = 10  # Можно изменить (больше = ближе)
                        map_url = f"https://www.arcgis.com/apps/mapviewer/index.html?center={center_y},{center_x}&level={zoom_level}&layers={FEATURE_LAYER_ID}"
                        print(f"\n1. Map Viewer (с автоматическим масштабированием):")
                        print(f"   {map_url}")
                        
                        # Ссылка 2: Через item с параметрами
                        item_url = f"https://www.arcgis.com/home/item.html?id={FEATURE_LAYER_ID}"
                        print(f"\n2. Item страница (откройте и нажмите 'Open in Map Viewer'):")
                        print(f"   {item_url}")
                        
                        # Ссылка 3: Прямая ссылка на Feature Service
                        if 'item' in globals():
                            print(f"\n3. Feature Service URL:")
                            print(f"   {item.url}")
                        
                        print("\n" + "="*70)
                        print("💡 ИНСТРУКЦИЯ:")
                        print("="*70)
                        print("1. Скопируйте ссылку №1 выше")
                        print("2. Вставьте в браузер")
                        print("3. Войдите в ArcGIS (если потребуется)")
                        print("4. Карта автоматически приблизится к вашим данным!")
                        print("5. Если точки все еще не видны, попробуйте:")
                        print("   - Нажать 'Zoom to Layer' (Масштабировать к слою)")
                        print("   - Или вручную приблизить карту (колесико мыши)")
                        print("="*70)
                        
                except Exception as e:
                    print(f"\n⚠️ Не удалось получить границы: {e}")
                    print("\n💡 Попробуйте вручную:")
                    print("   1. Откройте карту")
                    print("   2. Нажмите на слой в списке слоев")
                    print("   3. Выберите 'Zoom to Layer'")
                    
        except Exception as e:
            print(f"\n❌ Ошибка при проверке данных: {e}")
            import traceback
            traceback.print_exc()

# Если все переменные есть, создаем веб-карту автоматически
if 'gis' in globals() and 'feature_layer' in globals():
    try:
        count = feature_layer.query(where="1=1", return_count_only=True)
        if count > 0:
            print("\n" + "="*70)
            print("🗺️ СОЗДАНИЕ ВЕБ-КАРТЫ С АВТОМАТИЧЕСКИМ МАСШТАБИРОВАНИЕМ")
            print("="*70)
            
            from arcgis.mapping import WebMap
            
            # Создаем веб-карту
            webmap = WebMap()
            
            # Добавляем слой
            webmap.add_layer(feature_layer)
            
            # Получаем границы для установки начального экстента
            try:
                extent_result = feature_layer.query(where="1=1", return_extent_only=True)
                if extent_result and 'extent' in extent_result:
                    extent = extent_result['extent']
                    # Устанавливаем начальный экстент карты
                    webmap.extent = extent
            except:
                pass
            
            # Сохраняем карту
            webmap_item = webmap.save({
                "title": "ArcGIS Connector - Transformed Data Map",
                "tags": "wide-to-long, google-sheets, data-transformation",
                "description": "Карта с трансформированными данными из Google Sheets. Автоматически создана с правильным масштабированием."
            })
            
            print(f"\n✅ ВЕБ-КАРТА СОЗДАНА!")
            print(f"\n📍 Ссылка на карту: {webmap_item.url}")
            print(f"\n🆔 ID карты: {webmap_item.id}")
            print(f"\n🔗 Прямая ссылка: https://www.arcgis.com/home/item.html?id={webmap_item.id}")
            print(f"\n🗺️ Открыть в Map Viewer: https://www.arcgis.com/apps/mapviewer/index.html?webmap={webmap_item.id}")
            print("\n💡 Скопируйте последнюю ссылку - она откроет карту с правильным масштабом!")
            
    except Exception as e:
        print(f"\n⚠️ Не удалось создать веб-карту автоматически: {e}")
        print("\n💡 Используйте ссылки выше для ручного открытия")


## 🔗 ПОЛУЧИТЬ РАБОЧУЮ ССЫЛКУ НА КАРТУ

Выполните эту ячейку, чтобы получить прямую рабочую ссылку на карту с вашими данными


In [ ]:
# ПРОСТАЯ ЯЧЕЙКА ДЛЯ ПОЛУЧЕНИЯ РАБОЧЕЙ ССЫЛКИ

FEATURE_LAYER_ID = '6f8eb77db2dc41cc8261eb836e4b35af'

print("="*70)
print("🔗 РАБОЧИЕ ССЫЛКИ НА КАРТУ")
print("="*70)

# Проверяем, есть ли подключение и данные
if 'gis' in globals() and 'feature_layer' in globals():
    try:
        # Проверяем количество данных
        count = feature_layer.query(where="1=1", return_count_only=True)
        print(f"\n✅ Данные в слое: {count} точек")
        
        if count > 0:
            # Получаем границы для правильного масштабирования
            try:
                extent = feature_layer.query(where="1=1", return_extent_only=True)
                if extent and 'extent' in extent:
                    ext = extent['extent']
                    center_x = (ext.get('xmin', 0) + ext.get('xmax', 0)) / 2
                    center_y = (ext.get('ymin', 0) + ext.get('ymax', 0)) / 2
                    
                    # Создаем ссылку с правильным центром и зумом
                    map_link = f"https://www.arcgis.com/apps/mapviewer/index.html?center={center_y},{center_x}&level=12&layers={FEATURE_LAYER_ID}"
                    
                    print("\n" + "="*70)
                    print("📍 ССЫЛКА №1 (РЕКОМЕНДУЕТСЯ - с автоматическим масштабированием):")
                    print("="*70)
                    print(map_link)
                    print("="*70)
            except:
                pass
            
            # Создаем веб-карту автоматически
            try:
                from arcgis.mapping import WebMap
                
                print("\n🗺️ Создаю веб-карту...")
                webmap = WebMap()
                webmap.add_layer(feature_layer)
                
                # Устанавливаем правильный экстент
                try:
                    extent = feature_layer.query(where="1=1", return_extent_only=True)
                    if extent and 'extent' in extent:
                        webmap.extent = extent['extent']
                except:
                    pass
                
                # Сохраняем карту
                webmap_item = webmap.save({
                    "title": "ArcGIS Connector - Data Map",
                    "tags": "wide-to-long, google-sheets",
                    "description": "Карта с трансформированными данными"
                })
                
                print("\n" + "="*70)
                print("✅ ВЕБ-КАРТА СОЗДАНА!")
                print("="*70)
                print(f"\n📍 ССЫЛКА №2 (Веб-карта с правильным масштабом):")
                print(f"https://www.arcgis.com/apps/mapviewer/index.html?webmap={webmap_item.id}")
                print("\n" + "="*70)
                print("💡 СКОПИРУЙТЕ ЭТУ ССЫЛКУ И ОТКРОЙТЕ В БРАУЗЕРЕ!")
                print("="*70)
                
            except Exception as e:
                print(f"\n⚠️ Не удалось создать веб-карту: {e}")
        else:
            print("\n❌ В слое нет данных! Выполните ячейку 'Шаг 8: Загрузка данных'")
            
    except Exception as e:
        print(f"\n❌ Ошибка: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n❌ Не подключено к ArcGIS или Feature Layer не загружен")
    print("💡 Выполните ячейки:")
    print("   - 'Шаг 5: Подключение к ArcGIS'")
    print("   - 'Шаг 6: Получение Feature Layer'")

# Всегда показываем прямую ссылку через ID
print("\n" + "="*70)
print("📍 ССЫЛКА №3 (Прямая ссылка через ID):")
print("="*70)
print(f"https://www.arcgis.com/home/item.html?id={FEATURE_LAYER_ID}")
print("\n💡 Откройте эту ссылку → нажмите 'Open in Map Viewer'")
print("="*70)


## ⚠️ РЕШЕНИЕ: Feature Layer не найден или недоступен

Если вы видите ошибку "The item you requested cannot be found", выполните эту ячейку для создания нового Feature Layer


In [ ]:
# РЕШЕНИЕ ПРОБЛЕМЫ: Feature Layer не найден

print("="*70)
print("🔧 РЕШЕНИЕ: Создание нового Feature Layer")
print("="*70)

# Проверяем подключение
if 'gis' not in globals():
    print("\n❌ Не подключено к ArcGIS")
    print("💡 Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")
else:
    print(f"\n✅ Подключено к ArcGIS: {gis.properties.name}")
    
    # Проверяем, есть ли данные для загрузки
    if 'df_final' not in globals():
        print("\n❌ Данные не подготовлены")
        print("💡 Выполните ячейки до 'Шаг 4: Подготовка данных'")
    else:
        print(f"\n✅ Данные подготовлены: {len(df_final)} строк")
        
        # Создаем новый Feature Layer
        try:
            from arcgis.features import FeatureSet, FeatureCollection
            from arcgis.geometry import Point
            
            print("\n🗺️ Создаю новый Feature Layer...")
            
            # Создаем features из DataFrame
            features_list = []
            for idx, row in df_final.iterrows():
                try:
                    # Создаем геометрию
                    point = Point({
                        "x": float(row['long']),
                        "y": float(row['lat']),
                        "spatialReference": {"wkid": 4326}
                    })
                    
                    # Создаем атрибуты
                    attrs = {}
                    for field in ['d_date', 't_region', 't_city']:
                        if field in row.index:
                            attrs[field] = str(row[field]) if pd.notna(row[field]) else None
                    
                    for i in range(1, 11):
                        field = f'i_value_{i}'
                        if field in row.index:
                            try:
                                attrs[field] = int(float(row[field])) if pd.notna(row[field]) else 0
                            except:
                                attrs[field] = 0
                    
                    # Создаем feature
                    from arcgis.features import Feature
                    feature = Feature(geometry=point, attributes=attrs)
                    features_list.append(feature)
                    
                except Exception as e:
                    if idx < 3:  # Показываем только первые ошибки
                        print(f"⚠️ Ошибка в строке {idx}: {e}")
                    continue
            
            print(f"✅ Создано {len(features_list)} features")
            
            # Создаем Feature Collection
            feature_collection = FeatureCollection(features_list)
            
            # Создаем новый Feature Layer (Hosted Feature Layer)
            print("\n💾 Сохраняю Feature Layer в ArcGIS...")
            
            item_properties = {
                "title": "ArcGIS Connector - Transformed Data",
                "tags": "wide-to-long, google-sheets, data-transformation",
                "description": "Feature Layer с трансформированными данными из Google Sheets (Wide to Long)",
                "type": "Feature Service"
            }
            
            # Публикуем Feature Layer
            new_item = feature_collection.save(item_properties)
            
            print("\n" + "="*70)
            print("✅ НОВЫЙ FEATURE LAYER СОЗДАН!")
            print("="*70)
            print(f"\n📍 Item ID: {new_item.id}")
            print(f"\n🔗 Ссылка на Item: {new_item.url}")
            print(f"\n🗺️ Открыть в Map Viewer:")
            print(f"https://www.arcgis.com/apps/mapviewer/index.html?layers={new_item.id}")
            
            # Получаем Feature Layer
            if hasattr(new_item, 'layers') and len(new_item.layers) > 0:
                new_feature_layer = new_item.layers[0]
                print(f"\n✅ Feature Layer готов к использованию")
                print(f"\n📊 Количество features: {len(features_list)}")
                
                # Сохраняем ID для дальнейшего использования
                print(f"\n💡 Сохраните этот ID для дальнейшей работы: {new_item.id}")
                
                # Создаем веб-карту с правильным масштабированием
                try:
                    from arcgis.mapping import WebMap
                    
                    print("\n🗺️ Создаю веб-карту...")
                    webmap = WebMap()
                    webmap.add_layer(new_feature_layer)
                    
                    # Устанавливаем экстент на основе данных
                    try:
                        extent = new_feature_layer.query(where="1=1", return_extent_only=True)
                        if extent and 'extent' in extent:
                            webmap.extent = extent['extent']
                    except:
                        pass
                    
                    # Сохраняем веб-карту
                    webmap_item = webmap.save({
                        "title": "ArcGIS Connector - Data Map",
                        "tags": "wide-to-long, google-sheets",
                        "description": "Карта с трансформированными данными"
                    })
                    
                    print("\n" + "="*70)
                    print("✅ ВЕБ-КАРТА СОЗДАНА!")
                    print("="*70)
                    print(f"\n📍 ССЫЛКА НА КАРТУ (РАБОЧАЯ!):")
                    print(f"https://www.arcgis.com/apps/mapviewer/index.html?webmap={webmap_item.id}")
                    print("\n" + "="*70)
                    print("💡 СКОПИРУЙТЕ ЭТУ ССЫЛКУ И ОТКРОЙТЕ В БРАУЗЕРЕ!")
                    print("="*70)
                    
                except Exception as e:
                    print(f"\n⚠️ Не удалось создать веб-карту: {e}")
                    print(f"\n💡 Используйте прямую ссылку на Feature Layer выше")
            else:
                print("\n⚠️ Feature Layer не найден в item")
                
        except Exception as e:
            print(f"\n❌ Ошибка при создании Feature Layer: {e}")
            import traceback
            traceback.print_exc()
            print("\n💡 Альтернатива: Используйте ArcGIS Online для создания Feature Layer вручную")


## ✅ БЫСТРОЕ РЕШЕНИЕ: Создать новый Feature Layer и получить рабочую ссылку

**Выполните эту ячейку ПОСЛЕ того, как выполнили все шаги до "Шаг 4: Подготовка данных"**


In [ ]:
# БЫСТРОЕ РЕШЕНИЕ: Создание нового Feature Layer с рабочей ссылкой

print("="*70)
print("🚀 СОЗДАНИЕ НОВОГО FEATURE LAYER И КАРТЫ")
print("="*70)

# Проверяем необходимые условия
if 'gis' not in globals():
    print("\n❌ ОШИБКА: Не подключено к ArcGIS")
    print("💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")
elif 'df_final' not in globals():
    print("\n❌ ОШИБКА: Данные не подготовлены")
    print("💡 РЕШЕНИЕ: Выполните ячейки до 'Шаг 4: Подготовка данных'")
else:
    print(f"\n✅ Подключено к ArcGIS: {gis.properties.name}")
    print(f"✅ Данные подготовлены: {len(df_final)} строк")
    
    try:
        from arcgis.features import Feature, FeatureCollection
        from arcgis.geometry import Point
        
        print("\n📦 Создаю features из данных...")
        features_list = []
        
        for idx, row in df_final.iterrows():
            try:
                # Проверяем наличие координат
                if pd.isna(row.get('long')) or pd.isna(row.get('lat')):
                    continue
                
                # Создаем точку
                point = Point({
                    "x": float(row['long']),
                    "y": float(row['lat']),
                    "spatialReference": {"wkid": 4326}
                })
                
                # Создаем атрибуты
                attrs = {}
                
                # Текстовые поля
                for field in ['d_date', 't_region', 't_city']:
                    if field in row.index:
                        val = row[field]
                        attrs[field] = str(val) if pd.notna(val) and val != '' else None
                
                # Числовые поля (Значення 1-10)
                for i in range(1, 11):
                    field = f'i_value_{i}'
                    if field in row.index:
                        try:
                            val = row[field]
                            attrs[field] = int(float(val)) if pd.notna(val) and val != '' else 0
                        except:
                            attrs[field] = 0
                
                # Создаем feature
                feature = Feature(geometry=point, attributes=attrs)
                features_list.append(feature)
                
            except Exception as e:
                if len(features_list) < 3:  # Показываем только первые ошибки
                    print(f"⚠️ Ошибка в строке {idx}: {e}")
                continue
        
        print(f"✅ Создано {len(features_list)} features")
        
        if len(features_list) == 0:
            print("\n❌ ОШИБКА: Не удалось создать ни одного feature")
            print("💡 Проверьте, что в данных есть координаты (long, lat)")
        else:
            # Создаем Feature Collection
            print("\n💾 Публикую Feature Layer в ArcGIS...")
            feature_collection = FeatureCollection(features_list)
            
            # Сохраняем как Hosted Feature Layer
            new_item = feature_collection.save({
                "title": "ArcGIS Connector - Transformed Data",
                "tags": "wide-to-long, google-sheets, data-transformation",
                "description": "Feature Layer с трансформированными данными из Google Sheets"
            })
            
            print("\n" + "="*70)
            print("✅ FEATURE LAYER СОЗДАН!")
            print("="*70)
            print(f"\n📍 Item ID: {new_item.id}")
            print(f"🔗 Item URL: {new_item.url}")
            
            # Получаем Feature Layer
            if hasattr(new_item, 'layers') and len(new_item.layers) > 0:
                new_feature_layer = new_item.layers[0]
                
                # Создаем веб-карту
                print("\n🗺️ Создаю веб-карту...")
                from arcgis.mapping import WebMap
                
                webmap = WebMap()
                webmap.add_layer(new_feature_layer)
                
                # Устанавливаем правильный экстент
                try:
                    extent_result = new_feature_layer.query(where="1=1", return_extent_only=True)
                    if extent_result and 'extent' in extent_result:
                        webmap.extent = extent_result['extent']
                except:
                    pass
                
                # Сохраняем веб-карту
                webmap_item = webmap.save({
                    "title": "ArcGIS Connector - Data Map",
                    "tags": "wide-to-long, google-sheets",
                    "description": "Карта с трансформированными данными"
                })
                
                print("\n" + "="*70)
                print("✅ ВЕБ-КАРТА СОЗДАНА!")
                print("="*70)
                print(f"\n📍 ССЫЛКА НА КАРТУ (РАБОЧАЯ!):")
                print(f"https://www.arcgis.com/apps/mapviewer/index.html?webmap={webmap_item.id}")
                print("\n" + "="*70)
                print("💡 СКОПИРУЙТЕ ЭТУ ССЫЛКУ И ОТКРОЙТЕ В БРАУЗЕРЕ!")
                print("="*70)
                
                # Дополнительные ссылки
                print(f"\n📋 Дополнительные ссылки:")
                print(f"   - Item страница: {webmap_item.url}")
                print(f"   - Feature Layer: {new_item.url}")
                print(f"   - Map Viewer (через layer): https://www.arcgis.com/apps/mapviewer/index.html?layers={new_item.id}")
                
            else:
                print("\n⚠️ Feature Layer не найден в item")
                print(f"\n💡 Попробуйте открыть: {new_item.url}")
                
    except Exception as e:
        print(f"\n❌ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()
        print("\n💡 Альтернатива:")
        print("   1. Откройте https://www.arcgis.com/apps/mapviewer/index.html")
        print("   2. Войдите в свой аккаунт")
        print("   3. Нажмите 'Add' → 'Add Layer from File'")
        print("   4. Экспортируйте df_final в CSV и загрузите")


## 🔍 ПРОСТАЯ ДИАГНОСТИКА: Что не работает?

**Выполните эту ячейку, чтобы понять, на каком этапе проблема**


In [ ]:
# ПРОСТАЯ ДИАГНОСТИКА - что работает, что нет

print("="*70)
print("🔍 ДИАГНОСТИКА ПРОЕКТА")
print("="*70)

# Шаг 1: Проверка импортов
print("\n1️⃣ ПРОВЕРКА БИБЛИОТЕК:")
try:
    import pandas as pd
    print("   ✅ pandas")
except:
    print("   ❌ pandas - установите: pip install pandas")

try:
    import gspread
    print("   ✅ gspread")
except:
    print("   ❌ gspread - установите: pip install gspread")

try:
    from arcgis.gis import GIS
    print("   ✅ arcgis")
except:
    print("   ❌ arcgis - установите: pip install arcgis")

# Шаг 2: Проверка подключения к ArcGIS
print("\n2️⃣ ПРОВЕРКА ПОДКЛЮЧЕНИЯ К ARCGIS:")
if 'gis' in globals():
    try:
        print(f"   ✅ Подключено: {gis.properties.name}")
        print(f"   👤 Пользователь: {gis.users.me.username}")
    except:
        print("   ⚠️ Переменная gis есть, но подключение не работает")
else:
    print("   ❌ НЕ подключено к ArcGIS")
    print("   💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")

# Шаг 3: Проверка данных из Google Sheets
print("\n3️⃣ ПРОВЕРКА ДАННЫХ ИЗ GOOGLE SHEETS:")
if 'df' in globals():
    print(f"   ✅ Данные загружены: {len(df)} строк")
    print(f"   📋 Колонки: {list(df.columns)[:5]}...")
elif 'df_final' in globals():
    print(f"   ✅ Данные подготовлены: {len(df_final)} строк")
    print(f"   📋 Колонки: {list(df_final.columns)[:5]}...")
else:
    print("   ❌ Данные НЕ загружены")
    print("   💡 РЕШЕНИЕ: Выполните ячейки:")
    print("      - 'Шаг 1: Настройка доступа к Google Sheets'")
    print("      - 'Шаг 2: Чтение данных'")

# Шаг 4: Проверка Feature Layer
print("\n4️⃣ ПРОВЕРКА FEATURE LAYER:")
if 'feature_layer' in globals():
    try:
        count = feature_layer.query(where="1=1", return_count_only=True)
        print(f"   ✅ Feature Layer найден")
        print(f"   📊 Количество точек в слое: {count}")
        if count == 0:
            print("   ⚠️ ВНИМАНИЕ: Слой пустой - данные не загружены!")
            print("   💡 РЕШЕНИЕ: Выполните ячейку 'Шаг 8: Загрузка данных'")
    except Exception as e:
        print(f"   ⚠️ Feature Layer есть, но ошибка при запросе: {e}")
else:
    print("   ❌ Feature Layer НЕ найден")
    if 'gis' in globals():
        print("   💡 РЕШЕНИЕ: Выполните ячейку '✅ БЫСТРОЕ РЕШЕНИЕ' (создаст новый слой)")

# Шаг 5: Итоговая рекомендация
print("\n" + "="*70)
print("📋 ЧТО ДЕЛАТЬ ДАЛЬШЕ:")
print("="*70)

if 'gis' not in globals():
    print("\n❌ НАЧНИТЕ С ЭТОГО:")
    print("   1. Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")
    print("   2. Войдите в ArcGIS через браузер")
elif 'df_final' not in globals() and 'df' not in globals():
    print("\n❌ НАЧНИТЕ С ЭТОГО:")
    print("   1. Выполните ячейки для загрузки данных из Google Sheets")
    print("   2. Выполните трансформацию Wide to Long")
elif 'df_final' in globals() and 'gis' in globals():
    print("\n✅ ВСЁ ГОТОВО ДЛЯ СОЗДАНИЯ КАРТЫ!")
    print("\n💡 ВЫПОЛНИТЕ ЭТУ ЯЧЕЙКУ:")
    print("   '✅ БЫСТРОЕ РЕШЕНИЕ: Создать новый Feature Layer и получить рабочую ссылку'")
    print("\n   Эта ячейка:")
    print("   - Создаст Feature Layer с вашими данными")
    print("   - Создаст веб-карту")
    print("   - Выведет РАБОЧУЮ ссылку на карту")
else:
    print("\n⚠️ Частично готово - проверьте сообщения выше")

print("\n" + "="*70)


## Диагностика: Проверка загруженных данных


In [ ]:
# Проверяем, что данные действительно загружены в Feature Layer
print("="*60)
print("🔍 ДИАГНОСТИКА ЗАГРУЖЕННЫХ ДАННЫХ")
print("="*60)

try:
    # Проверяем подключение к ArcGIS
    if 'gis' not in globals():
        print("❌ Переменная 'gis' не найдена. Выполните ячейку 'Шаг 5: Подключение к ArcGIS'")
    else:
        print(f"✅ Подключение к ArcGIS: {gis.properties.name}")
    
    # Проверяем Feature Layer
    if 'feature_layer' not in globals():
        print("❌ Переменная 'feature_layer' не найдена. Выполните ячейку 'Шаг 6: Получение Feature Layer'")
    else:
        # Проверяем количество features
        count = feature_layer.query(where="1=1", return_count_only=True)
        print(f"✅ Feature Layer найден")
        print(f"📊 Количество features в слое: {count}")
        
        if count == 0:
            print("\n⚠️ ВНИМАНИЕ: В слое нет данных!")
            print("Возможные причины:")
            print("1. Данные не были загружены (не выполнена ячейка 'Шаг 8: Загрузка данных')")
            print("2. Данные были удалены")
            print("3. Нет прав доступа к данным")
            print("\n💡 Решение: Выполните ячейку 'Шаг 8: Загрузка данных в Feature Layer'")
        else:
            # Получаем примеры данных
            print(f"\n✅ Данные загружены успешно!")
            sample = feature_layer.query(where="1=1", return_geometry=True, result_record_count=3)
            print(f"\nПримеры загруженных точек:")
            for i, feature in enumerate(sample.features[:3], 1):
                geom = feature.geometry
                attrs = feature.attributes
                print(f"\n{i}. Координаты: ({geom.get('x', 'N/A')}, {geom.get('y', 'N/A')})")
                print(f"   Регион: {attrs.get('t_region', 'N/A')}")
                print(f"   Город: {attrs.get('t_city', 'N/A')}")
                print(f"   Дата: {attrs.get('d_date', 'N/A')}")
            
            # Проверяем границы данных для масштабирования карты
            extent = feature_layer.query(where="1=1", return_extent_only=True)
            if extent:
                print(f"\n📍 Границы данных:")
                print(f"   Запад: {extent.get('xmin', 'N/A')}")
                print(f"   Восток: {extent.get('xmax', 'N/A')}")
                print(f"   Юг: {extent.get('ymin', 'N/A')}")
                print(f"   Север: {extent.get('ymax', 'N/A')}")
                
                print(f"\n💡 Если карта показывает весь мир, попробуйте:")
                print(f"   1. Найти кнопку 'Zoom to Layer' или 'Масштабировать к слою'")
                print(f"   2. Или вручную приблизить карту к координатам выше")
        
        # Проверяем URL слоя
        if 'item' in globals():
            print(f"\n🔗 Ссылка на Feature Layer:")
            print(f"   {item.url}")
            print(f"\n📋 Инструкция по добавлению на карту:")
            print(f"   1. Откройте: https://www.arcgis.com/apps/mapviewer/index.html")
            print(f"   2. Нажмите 'Add' → 'Browse Layers'")
            print(f"   3. В поиске введите: {item.id}")
            print(f"   4. Или найдите в 'My Content' → 'My Layers'")
            print(f"   5. Добавьте слой на карту")
            print(f"   6. Нажмите на слой → 'Zoom to Layer' для приближения")
            
except Exception as e:
    print(f"❌ Ошибка при проверке: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*60)


## Шаг 11: Создание веб-карты с данными

Создадим веб-карту и добавим наш Feature Layer, чтобы сразу видеть точки на карте


In [ ]:
from arcgis.mapping import WebMap
from arcgis.mapping import MapImageLayer

# Проверяем, что данные загружены
try:
    count = feature_layer.query(where="1=1", return_count_only=True)
    print(f"✅ В слое {count} features")
    
    if count > 0:
        # Создаем веб-карту
        webmap = WebMap()
        
        # Добавляем наш Feature Layer на карту
        webmap.add_layer(feature_layer)
        
        # Сохраняем карту
        webmap_item = webmap.save(
            {
                "title": "ArcGIS Connector - Transformed Data",
                "tags": "wide-to-long, google-sheets, data-transformation",
                "description": "Карта с трансформированными данными из Google Sheets"
            }
        )
        
        print("\n" + "="*60)
        print("✅ ВЕБ-КАРТА СОЗДАНА!")
        print("="*60)
        print(f"\n📍 Ссылка на карту: {webmap_item.url}")
        print(f"\n🆔 ID карты: {webmap_item.id}")
        print(f"\n🔗 Прямая ссылка: https://www.arcgis.com/home/item.html?id={webmap_item.id}")
        print(f"\n🗺️ Открыть в Map Viewer: https://www.arcgis.com/apps/mapviewer/index.html?webmap={webmap_item.id}")
        print("\n" + "="*60)
        print("💡 Скопируйте любую ссылку выше и откройте в браузере!")
        print("="*60)
    else:
        print("⚠️ В слое нет данных. Убедитесь, что данные были загружены.")
        
except Exception as e:
    print(f"❌ Ошибка: {e}")
    print("\nПопробуйте добавить слой вручную:")
    print(f"1. Откройте: https://www.arcgis.com/apps/mapviewer/index.html")
    print(f"2. Нажмите 'Add' → 'Browse Layers'")
    print(f"3. Найдите слой с ID: 6f8eb77db2dc41cc8261eb836e4b35af")
    print(f"4. Добавьте его на карту")


## Альтернатива: Добавить слой вручную (если автоматическое создание не сработало)

Если веб-карта не создалась автоматически, добавьте слой вручную:


In [ ]:
print("="*60)
print("📋 ИНСТРУКЦИЯ: Как добавить слой вручную")
print("="*60)
print("""
1. Откройте ArcGIS Map Viewer:
   https://www.arcgis.com/apps/mapviewer/index.html

2. Нажмите кнопку "Add" (Добавить) в верхней панели

3. Выберите "Browse Layers" (Обзор слоев)

4. В поиске введите ID слоя:
   6f8eb77db2dc41cc8261eb836e4b35af

5. Или найдите слой в разделе "My Content" (Мой контент)

6. Нажмите "Add" рядом с нужным слоем

7. Слой появится на карте с вашими точками!

8. При клике на точку вы увидите popup с данными
""")
print("="*60)

# Также показываем прямую ссылку на Feature Layer
print(f"\n🔗 Прямая ссылка на Feature Layer:")
print(f"https://www.arcgis.com/home/item.html?id=6f8eb77db2dc41cc8261eb836e4b35af")
print("\nОткройте эту ссылку → нажмите 'Open in Map Viewer'")


## Шаг 5: Подключение к ArcGIS

**Варианты доступа (выберите один):**

1. **OAuth через браузер** (рекомендуется) - просто войдите в свой ArcGIS аккаунт
2. **API Key** - если у вас есть ключ
3. **Username/Password** - если знаете URL портала


In [ ]:
# ВАРИАНТ 1: OAuth (откроется браузер для авторизации) - РЕКОМЕНДУЕТСЯ
# Это самый простой способ - просто войдите в свой ArcGIS аккаунт
gis = GIS()  # Откроет браузер для входа

# ВАРИАНТ 2: Если уже авторизованы, можно использовать профиль
# gis = GIS(profile='your_profile_name')

# ВАРИАНТ 3: Прямой вход (если знаете URL вашего портала)
# gis = GIS("https://your-org.maps.arcgis.com", "username", "password")

# ВАРИАНТ 4: API Key (если есть)
# gis = GIS(api_key='YOUR_API_KEY')

print(f"✅ Подключено к: {gis.properties.name}")
print(f"Пользователь: {gis.users.me.username}")


## Шаг 6: Получение Feature Layer по ID


In [ ]:
# ID Feature Layer из ТЗ
FEATURE_LAYER_ID = '6f8eb77db2dc41cc8261eb836e4b35af'

# Получаем item по ID
item = gis.content.get(FEATURE_LAYER_ID)
print(f"✅ Item найден: {item.title}")
print(f"Type: {item.type}")

# Получаем Feature Layer
if hasattr(item, 'layers') and len(item.layers) > 0:
    feature_layer = item.layers[0]
else:
    feature_layer = item

print(f"✅ Feature Layer готов к работе")

# Проверяем схему полей
print("\nПоля в Feature Layer:")
for field in feature_layer.properties.fields:
    print(f"  - {field['name']} ({field['type']})")


## Шаг 7: Подготовка Features для загрузки


In [ ]:
# Функция для создания Feature из строки DataFrame
def create_feature(row):
    # Создаем геометрию (точка) из координат
    try:
        geometry = {
            "x": float(row['long']),
            "y": float(row['lat']),
            "spatialReference": {"wkid": 4326}  # WGS84
        }
    except (ValueError, KeyError) as e:
        print(f"Ошибка координат: {e}")
        return None
    
    # Создаем атрибуты
    attributes = {}
    for field in field_mapping.keys():
        if field in row.index:
            value = row[field]
            # Преобразуем типы данных
            if field.startswith('i_value_'):
                try:
                    attributes[field] = int(float(value)) if pd.notna(value) and value != '' else 0
                except:
                    attributes[field] = 0
            elif field == 'd_date':
                # Преобразуем дату в timestamp (миллисекунды)
                if pd.notna(value) and value != '':
                    try:
                        dt = pd.to_datetime(value)
                        attributes[field] = int(dt.timestamp() * 1000)
                    except:
                        attributes[field] = None
                else:
                    attributes[field] = None
            else:
                attributes[field] = str(value) if pd.notna(value) and value != '' else None
    
    return {
        "geometry": geometry,
        "attributes": attributes
    }

# Создаем список features
features = []
errors = 0

for idx, row in df_final.iterrows():
    try:
        feature = create_feature(row)
        if feature:
            features.append(feature)
    except Exception as e:
        errors += 1
        if errors <= 5:  # Показываем только первые 5 ошибок
            print(f"Ошибка в строке {idx}: {e}")

print(f"✅ Создано {len(features)} features для загрузки")
if errors > 0:
    print(f"⚠️ Ошибок: {errors}")


In [ ]:
# ОПЦИОНАЛЬНО: Очистка существующих данных
# Раскомментируйте следующую строку, если нужно удалить все существующие features
# feature_layer.delete_features(where="1=1")
# print("Существующие данные удалены")

# Добавляем features батчами (по 1000 за раз для надежности)
batch_size = 1000
total_added = 0
total_errors = 0

for i in range(0, len(features), batch_size):
    batch = features[i:i+batch_size]
    try:
        result = feature_layer.edit_features(adds=batch)
        if result['addResults']:
            success_count = sum(1 for r in result['addResults'] if r.get('success', False))
            error_count = len(batch) - success_count
            total_added += success_count
            total_errors += error_count
            print(f"Батч {i//batch_size + 1}: добавлено {success_count}/{len(batch)} features")
            if error_count > 0:
                print(f"  ⚠️ Ошибок в батче: {error_count}")
    except Exception as e:
        print(f"❌ Ошибка при добавлении батча {i//batch_size + 1}: {e}")
        total_errors += len(batch)

print(f"\n✅ Всего добавлено: {total_added} features")
if total_errors > 0:
    print(f"⚠️ Всего ошибок: {total_errors}")


## Шаг 9: Проверка результата


In [ ]:
# Проверяем количество features в слое
query_result = feature_layer.query(where="1=1", return_count_only=True)
print(f"✅ Всего features в слое: {query_result}")

# Получаем несколько примеров
sample = feature_layer.query(where="1=1", return_geometry=True, result_record_count=5)
print("\nПримеры features:")
for i, feature in enumerate(sample.features[:3], 1):
    print(f"\n{i}. Атрибуты:")
    for key, value in feature.attributes.items():
        if value is not None:
            print(f"   {key}: {value}")

print("\n" + "="*50)
print("✅ Данные успешно загружены в ArcGIS Feature Layer!")
print("="*50)
print(f"\nСсылка на карту: {item.url}")
